In [736]:
import pandas as pd
import numpy as np
import collections
import operator
import re

In [737]:
## Here goes the implementation for predicing haplogroups

In [738]:
df_intermediate = pd.read_csv("Hg_Prediction_tables/Intermediates.txt", header=None)
intermediates = df_intermediate[0].values

In [739]:
df_haplogroup = pd.read_csv("Output_example_datasets/Modern_DNA_WGS/HG003/HG003.out", sep="\t")

In [740]:
df_haplogroup = df_haplogroup.sort_values(by=['haplogroup'])

### Derive State

#### Removes the intermediate haplogroups temporally

In [741]:
df_derived = df_haplogroup[df_haplogroup["state"] == "D"]

In [742]:
## instance with only D state
df_tmp = df_derived
for hg in intermediates:    
    df_tmp = df_tmp.drop(df_tmp[df_tmp.haplogroup == hg].index)
hg = df_tmp["haplogroup"].values

#### Choose the haplogroup based on the highest count in the first letter (E.x. J2a21 = J)

In [743]:
list_hg = []
for i in hg:
    list_hg.append(i[0])
collections_hg = collections.Counter(list_hg)

In [744]:
init_hg = max(collections_hg.items(), key=operator.itemgetter(1))[0]
tmp_init_hg = init_hg+"_int.txt"
hg_intermediate_file = "Hg_Prediction_tables/"+tmp_init_hg
df_intermediate = pd.read_csv(hg_intermediate_file, header=None, sep="\t")

### QC.1 
Now you have the putative haplogroup. Open the file with the haplogroup intermediate and show the intermediate haplogroups again and compare if there are some where does not match with the correct state and store it a error. Give the estimate of the  correct_count/total  

$$QC1 = \frac{correct state } {total}$$

where 
###### correct state is the total number of correct haplogroup state found in the intermediate haplogroup file
###### total of all intermediate haplogroup found in the intermediate haplogroup file

In [745]:
total = 0
correct_state = 0
for i in df_intermediate.values:
    tmp = df_derived.loc[df_derived["haplogroup"] == i[0]]
    if not tmp.empty:        
        correct_state += np.sum(i[1] == tmp["state"])                                                
        total += len(tmp)           
qc_one = (correct_state / total)

Removes all haplogroup but the main one
Check if the preffix of all main haplogroup with D state by allowing one haplogroup that does not match to the main haplogroup until you get back to the origin. 
If mismatch > 1 skip choose as main haplogroup the following in the largest length and continue. In case haplogroup contains ~ symbol at the end ignore it temporally for preffix comparison but if this is the main haplogroup store the special character at the end. 

In [746]:
list_main_hg = []
for i in hg:    
    if i.startswith(init_hg):
        list_main_hg.append(i)
list_main_hg = sorted(list(set(list_main_hg)), reverse=True)
putative_hg = ""
for i in range(len(list_main_hg)-1):
    mismatch = 0    
    for j in range(i,len(list_main_hg)):                
        current_hg = list_main_hg[i]            
        next_hg    = list_main_hg[j]
        current_hg = current_hg.replace("~","")
        next_hg    = next_hg.replace("~","")        
        if not next_hg in current_hg:
            mismatch += 1    
        if mismatch >= 2:
            break
    if mismatch < 2:
        putative_hg = list_main_hg[i]        
        break

### Ancestral and Derive State

### QC.2

Check if the same name of the main haplogroup appears as an Ancestral State and save the number of count and calculate QC2:

$$QC2 = \frac{Dhg-Ahg} {total}$$
where 
###### Dhg is the total number of derived main haplogroup 
###### Ahg is the total number of ancestral with same name of main haplogroup
###### total of all derived and ancestral haplogroup with the same name

In [747]:
total_qctwo = len(df_haplogroup.loc[df_haplogroup["haplogroup"] == putative_hg])
Ahg = np.sum("A" == df_haplogroup.loc[df_haplogroup["haplogroup"] == putative_hg]["state"])
qc_two = (total_qctwo-Ahg)/total_qctwo

### QC.3
Show both Ancestral and Derived states from the main haplogroup and check the preffix of each haplogroup that is before the main one until you reach the root. If there are some Ancestral states which follows the pattern/preffix from the main haplogroup keep the count of how many of these appear and this will give the total count. Substract the ones found from the corrected and this will give the QC.3 score. 

In [748]:
list_main_hg_all = []
list_hg_all = df_haplogroup["haplogroup"].values
for i in list_hg_all:    
    if i.startswith(init_hg):
        list_main_hg_all.append(i)
list_main_hg_all = sorted(list(set(list_main_hg_all)), reverse=True)
df_main_hg_all = df_haplogroup.loc[df_haplogroup["haplogroup"].isin(list_main_hg_all)]
df_main_hg_all = df_main_hg_all[["haplogroup","state","marker_name"]]
df_main_hg_all = df_main_hg_all.sort_values(by="haplogroup", ascending=False).values

In [749]:
a_match = 0
total_match = 0
for i in df_main_hg_all:
    tmp_hg = i[0].replace("~","")
    if tmp_hg in putative_hg:
        total_match +=1        
        if i[1] == "A":
            a_match += 1
qc_three = (total_match - a_match) / total_match

### Final output
1) show the haplogroup name 
    if main haplogroup is a preffix from a higher resolution Ancestral state haplogroup report this haplogroup and marker name (Ex. HG: J21a Marker: L123)
    
2) Not J21a-L123

Could be that that there are more than one contain as a preffix from an ancestral state haplogroup. Should report all of them only if there are different haplogroup name with the resolution
**Check this in more detail later on... 

In [750]:
df_putative_ancestral_hg = pd.DataFrame()
for i in df_main_hg_all:    
    if i[0] == putative_hg:
        break
    if putative_hg in i[0]:                        
        tmp = pd.DataFrame([i], columns=["haplogroup","state","marker"])        
        df_putative_ancestral_hg = df_putative_ancestral_hg.append(tmp)        
df_putative_ancestral_hg = df_putative_ancestral_hg.sort_values(by="haplogroup")
df_putative_ancestral_hg = df_putative_ancestral_hg
df_putative_ancestral_hg.index = range(len(df_putative_ancestral_hg))

In [751]:
putative_ancestral_hg = []
for i in range(len(df_putative_ancestral_hg)):        
    row = df_putative_ancestral_hg.iloc[i]
    if putative_ancestral_hg == []:        
        putative_ancestral_hg.append(row.values)                    
    else:        
        if putative_ancestral_hg[-1][0] not in row[0]:
            putative_ancestral_hg.append(row.values)                                  
putative_ancestral_hg = np.array(putative_ancestral_hg)

In [752]:
print(qc_one)
print(qc_two)
print(qc_three)
print(putative_hg)
for pahg in putative_ancestral_hg:
    print("Not {}".format(pahg[0][0]+"-"+pahg[2]))

1.0
1.0
0.9926470588235294
J1a2b3a1
Not J-L817
Not J-L819
Not J-L820
